In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_8/models/B5R2b5_AN_1FC_fold8_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold8.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4601, 25)
(4601, 25)
Normal:  (3006, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4601 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.17626584, -0.12658142,  0.2559714 , ..., -0.08333829,
         0.03707984, -0.09873207],
       [-0.09212791, -0.11146519,  0.22980545, ..., -0.05211022,
         0.11202876, -0.13193078],
       [-0.15425348, -0.12143129,  0.17639062, ..., -0.11484177,
         0.06949777, -0.07376634],
       ...,
       [ 0.54450727,  0.28678614, -0.20211351, ...,  0.55092329,
         0.02204835,  0.42374232],
       [ 0.36836827,  0.48132268, -0.20730859, ...,  0.53738129,
        -0.10754952,  0.95654106],
       [ 0.33467695,  0.86196464, -0.20322238, ...,  0.99841034,
        -0.18377063,  0.6787436 ]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.176266,-0.126581,0.255971,-0.094936,-0.178978,-0.142334,0.117486,-0.208209,-0.202585,0.032798,...,-0.134565,-0.111791,0.085412,-0.147788,-0.170082,0.055189,-0.041769,-0.083338,0.037080,-0.098732
1,-0.092128,-0.111465,0.229805,-0.127210,-0.173608,-0.140279,0.379899,-0.161369,-0.161904,0.088065,...,-0.110548,-0.021287,0.056410,-0.081879,-0.178106,0.080956,0.028838,-0.052110,0.112029,-0.131931
2,-0.154253,-0.121431,0.176391,-0.155759,-0.177797,-0.138466,0.364843,-0.213568,-0.190205,0.140837,...,-0.112427,0.133400,0.217222,-0.084821,-0.194163,-0.017772,0.024317,-0.114842,0.069498,-0.073766
3,-0.185132,-0.133850,0.273052,-0.063366,-0.163682,-0.063715,0.185061,-0.077128,-0.200565,0.005331,...,-0.106889,-0.005941,0.051541,-0.111749,-0.164659,0.158381,-0.075338,-0.134253,0.036917,-0.140047
4,-0.049047,-0.102947,0.456641,0.039560,-0.074768,-0.067078,0.402351,-0.136153,-0.167107,-0.009086,...,-0.083589,-0.006481,-0.023120,-0.112284,-0.117693,0.045899,0.044355,-0.117944,0.083893,-0.150556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.852471,0.619833,-0.210811,0.183871,0.413822,0.279697,-0.083906,-0.021640,0.879066,-0.167181,...,-0.034367,-0.158682,-0.087958,1.053598,0.755825,-0.198221,-0.093761,0.566742,-0.122881,0.493488
4597,0.314243,0.125983,-0.182202,0.771696,0.262492,0.364797,-0.169629,0.229504,0.178705,-0.081125,...,0.480747,-0.135182,-0.073453,0.803721,0.324225,-0.186114,0.005001,0.223349,-0.108289,0.200945
4598,0.544507,0.286786,-0.202114,0.683390,0.149633,0.473665,-0.068342,0.013771,0.227784,-0.152739,...,0.460210,-0.131679,-0.017504,0.552469,0.238670,-0.081911,0.036632,0.550923,0.022048,0.423742
4599,0.368368,0.481323,-0.207309,0.423160,0.633391,0.194087,-0.209065,0.566697,0.373197,-0.059514,...,0.266056,-0.191710,-0.069610,0.598990,0.284077,-0.150701,-0.122327,0.537381,-0.107550,0.956541


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.176266,-0.126581,0.255971,-0.094936,-0.178978,-0.142334,0.117486,-0.208209,-0.202585,0.032798,...,-0.170082,0.055189,-0.041769,-0.083338,0.037080,-0.098732,Abnormal,P31,FP-B,AB01
1,-0.092128,-0.111465,0.229805,-0.127210,-0.173608,-0.140279,0.379899,-0.161369,-0.161904,0.088065,...,-0.178106,0.080956,0.028838,-0.052110,0.112029,-0.131931,Abnormal,P1,FP-A,AB01
2,-0.154253,-0.121431,0.176391,-0.155759,-0.177797,-0.138466,0.364843,-0.213568,-0.190205,0.140837,...,-0.194163,-0.017772,0.024317,-0.114842,0.069498,-0.073766,Abnormal,P2,FP-A,AB01
3,-0.185132,-0.133850,0.273052,-0.063366,-0.163682,-0.063715,0.185061,-0.077128,-0.200565,0.005331,...,-0.164659,0.158381,-0.075338,-0.134253,0.036917,-0.140047,Abnormal,P42,FP-B,AB01
4,-0.049047,-0.102947,0.456641,0.039560,-0.074768,-0.067078,0.402351,-0.136153,-0.167107,-0.009086,...,-0.117693,0.045899,0.044355,-0.117944,0.083893,-0.150556,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.852471,0.619833,-0.210811,0.183871,0.413822,0.279697,-0.083906,-0.021640,0.879066,-0.167181,...,0.755825,-0.198221,-0.093761,0.566742,-0.122881,0.493488,Normal,P32,FP-B,Normal
4597,0.314243,0.125983,-0.182202,0.771696,0.262492,0.364797,-0.169629,0.229504,0.178705,-0.081125,...,0.324225,-0.186114,0.005001,0.223349,-0.108289,0.200945,Normal,P42,FP-B,Normal
4598,0.544507,0.286786,-0.202114,0.683390,0.149633,0.473665,-0.068342,0.013771,0.227784,-0.152739,...,0.238670,-0.081911,0.036632,0.550923,0.022048,0.423742,Normal,P52,FP-C,Normal
4599,0.368368,0.481323,-0.207309,0.423160,0.633391,0.194087,-0.209065,0.566697,0.373197,-0.059514,...,0.284077,-0.150701,-0.122327,0.537381,-0.107550,0.956541,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold8_1.csv') #เปลี่ยนชื่อไฟล์